In [ ]:
import os
import sys
import math
import warnings
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import display
warnings.simplefilter(action='ignore', category=FutureWarning)
excel_file_path = '/content/sample_data/8-9 to 10-11 Sep.xlsx'
xls = pd.ExcelFile(excel_file_path)
available_sheets = xls.sheet_names
Sheets = {
    "Azaad Digital": ["Azaad Anchors", "Azaad Anchors "],
    "Pukhtun Digital": ["Pukhtun Anchors", "Pakhtun Anchors"],
    "Kashmir Digital": ["Kashmir Anchors"],
    "Burak": ["Burak Anchors", "Burak Anchors "]
}
formattedVloggerData = pd.DataFrame(columns=["Name", "No of Vlogs", "Views", "Platform", "Channel", "Date"])
formattedReporterData = pd.DataFrame(columns=["Name", "No of Shows", "Views", "Channel", "Date"])
formattedAnchorData = pd.DataFrame(columns=["Name", "No of Shows", "Views", "Channel", "Date"])
formattedAnalystData = pd.DataFrame(columns=["Name", "No of Shows", "Views", "Channel", "Date"])
def dateLogicMaker3000(realSheet, designation, designationData, formattedDataFinalized):
    dd = designationData.iloc[::-1].reset_index(drop=True)
    dd.columns = dd.iloc[1]
    dd = dd.drop([0,1]).reset_index(drop=True)
    file_name = os.path.splitext(os.path.basename(excel_file_path))[0]
    allDates = file_name.split("o", 1)[1].strip()
    allDates = " ".join(allDates.split())
    dates = allDates.split(" ")[0].strip()
    month = allDates.split(" ")[1].strip()
    year = allDates.split(" ")[2].strip() if len(allDates.split(" ")) > 2 else "2025"
    dd.columns = dd.columns.str.strip()
    name = dd["Name"]
    date = dates.split("-")
    loopStoper = int(date[2].strip()) if len(date) > 2 else int(date[1].strip())
    selectedDayArray = dataExtractFormat[monthStartDay]
    selectedDayArrayUsed = False
    normalEnteryDate = [3, 2, 2]
    i = 1
    j = 0
    k = 0
    while i <= loopStoper:
        fullDate = f"{i}/{month}/{year}"
        cols = dd.columns.tolist()
        print(cols)
        keyColumnSelectors = ["Final Views", "Final Views ", "Final Views 48 Hrs", f"Final Views 48 Hrs({i} & {i+1})", "Total 48 hrs Views", f"48 hrs ({i}&{i+1})", "Views", "48 Hrs", f"48 Hrs ({i} & {i+1})", f"48 hrs ({i}&{i+1})", "48 Hrs Views", "48 Hrs Views ", "48 hrs"]
        for col in cols:
            found = False
            if col in keyColumnSelectors:
                found = True
                print(f"\nFOUNDED: Column 'Final Views' as {col} in {realSheet} starting from {fullDate}")
                idx = cols.index(col) + 1
                dataPerDate = dd.iloc[:, :idx]
                dd = dd.iloc[:, idx:]
                dataPerDate = dataPerDate.fillna(0)
                dataPerDate["Name"] = name
                if selectedDayArrayUsed == False:
                    if k == len(selectedDayArray):
                        selectedDayArrayUsed = True
                    else:
                        dataBreak = selectedDayArray[k]
                        if designation == "Vlogger":
                            formattedDataFinalized = enterDataInMyFormatVlogger(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        else:
                            formattedDataFinalized = enterDataInMyFormatJournalist(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + selectedDayArray[k]
                        k = k + 1
                if selectedDayArrayUsed == True:
                    if j == 0:
                        dataBreak = normalEnteryDate[j]
                        if designation == "Vlogger":
                            formattedDataFinalized = enterDataInMyFormatVlogger(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        else:
                            formattedDataFinalized = enterDataInMyFormatJournalist(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = j + 1
                    elif j == 1:
                        dataBreak = normalEnteryDate[j]
                        if designation == "Vlogger":
                            formattedDataFinalized = enterDataInMyFormatVlogger(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        else:
                            formattedDataFinalized = enterDataInMyFormatJournalist(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = j + 1
                    elif j == 2:
                        dataBreak = normalEnteryDate[j]
                        if designation == "Vlogger":
                            formattedDataFinalized = enterDataInMyFormatVlogger(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        else:
                            formattedDataFinalized = enterDataInMyFormatJournalist(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized)
                        i = i + normalEnteryDate[j]
                        j = 0
                        print("i have entered the data")
                    else:
                        print("ERROR: In using the normal sequence of date-wise entry")
                        break
                break
        if found == False:
            i = i + 1
        print("just gonna retirn now")
    return found, formattedDataFinalized

def enterDataInMyFormatVlogger(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized):
    keyPlatforms = {
        "Youtube": ["YT Views"],
        "Twitter (X)": ["Twitter Views"],
        "TikTok": ["TiktokViews", "Tiktok Views"],
        "Instagram": ["Instagram"],
        "Facebook":["FB Views"]
    }
    month_map = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
        "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
    }
    for platform, keyPlatformGroup in keyPlatforms.items():
        properDate = i
        keyPlatform = next((p for p in keyPlatformGroup if p in dataPerDate.columns), None)
        if "No of Vlogs" not in dataPerDate.columns:
            print(f"Column 'No of Vlogs' not found in {realSheet} for {fullDate}, so created it column with value 1 if there are views other wise 0")
            dataPerDate["No of Vlogs"] = np.where(dataPerDate[keyPlatform] > 1, 1, 0)
        if keyPlatform is None:
            print(f"NOT FOUND / MISMATCHED: Key platform column in {realSheet} for {platform} at date: {properDate}/{month}/{year}")
        else:
            dataPerDate[keyPlatform] = np.ceil(dataPerDate[keyPlatform] / dataBreak).astype(int)
            dataPerDate["No of Vlogs"] = np.ceil(dataPerDate["No of Vlogs"] / dataBreak).astype(int)
            for l in range(dataBreak):
                newData = pd.DataFrame({
                    "Name": dataPerDate["Name"],
                    "No of Vlogs": np.where(dataPerDate[keyPlatform] > 0, dataPerDate["No of Vlogs"], 0),
                    "Views": dataPerDate[keyPlatform],
                    "Platform": platform,
                    "Channel": realSheet,
                    "Date": datetime(int(year), month_map[month], int(properDate)),
                    "Designation": designation
                })
                formattedDataFinalized = pd.concat([formattedDataFinalized, newData], ignore_index=True)
                properDate = properDate + 1
    return formattedDataFinalized

def enterDataInMyFormatJournalist(realSheet, designation, fullDate, month, year, name, col, dataBreak, i, dataPerDate, formattedDataFinalized):
    properDate = i
    foundShowCol = None
    month_map = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
        "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
    }
    keyShows = ["No of Shows", "no of Shows"]
    for keyShow in keyShows:
        if keyShow in dataPerDate.columns:
            foundShowCol = keyShow
            break

    if foundShowCol is None:
        print(f"Column 'No of Shows' not found in {realSheet} for {fullDate}, so created it column with value 1 if there are views other wise 0")
        dataPerDate["No of Shows"] = np.where(dataPerDate[col] > 1, 1, 0)
    else:
        dataPerDate.rename(columns={foundShowCol: "No of Shows"}, inplace=True)
    dataPerDate[col] = np.ceil(dataPerDate[col] / dataBreak).astype(int)
    dataPerDate["No of Shows"] = np.ceil(dataPerDate["No of Shows"] / dataBreak).astype(int)
    for l in range(dataBreak):
        newData = pd.DataFrame({
            "Name": dataPerDate["Name"],
            "No of Shows": dataPerDate["No of Shows"],
            "Views": dataPerDate[col],
            "Channel": realSheet,
            "Date": datetime(int(year), month_map[month], int(properDate)),
            "Designation": designation
        })
        formattedDataFinalized = pd.concat([formattedDataFinalized, newData], ignore_index=True)
        properDate = properDate + 1
    return formattedDataFinalized

print("Enter the number according to the day, month started:")
print("Monday    : 1\nTuesday   : 2\nWednesday : 3\nThursday  : 4\nFriday    : 5\nSaturday  : 6\nSunday    : 7")
monthStartDay = input("Enter number: ")
dataExtractFormat = {
    "1": [2, 2],
    "2": [1, 2],
    "3": [2],
    "4": [1],
    "5": [],
    "6": [2, 2, 2],
    "7": [1, 2, 2]
}
for realSheet, sheetGroup in Sheets.items():
    existingSheet = next((s for s in sheetGroup if s in available_sheets), None)
    if existingSheet:
        df = pd.read_excel(xls, sheet_name=existingSheet)
        print(f"\n\nLOADED SHEET: {existingSheet} as {realSheet}")
        df = df.dropna(how="all").reset_index(drop=True)
        currentHeaders = df.columns.tolist()
        df.columns = df.iloc[0]
        df = pd.concat([pd.DataFrame([currentHeaders], columns=df.columns), df], ignore_index=True)
        vdInvert = df
        vdInvert = vdInvert.iloc[::-1].reset_index(drop=True)

        ##Vlogger Data Formation Starts
        print("Started: starting formatting of vloggers")
        names = vdInvert["Name"]
        names = names.to_list()
        cut = None
        for row, name in enumerate(names):
            if "Reporters / Vloggers" in name:
                cut = row + 1
                break
        if cut is not None:
            vloggerData = vdInvert.iloc[:cut].copy()
            otherData = vdInvert.iloc[cut:].copy()
        else:
            vloggerData = None
            otherData = vdInvert
            print(f"NOT FOUND: Reporters data as 'Reporters / Vloggers' in the {realSheet} for vlogger")
            break
        foundVlogger, formattedVloggerData = dateLogicMaker3000(realSheet, "Vlogger", vloggerData, formattedVloggerData)
        if foundVlogger == False:
            print(f"NOT FOUND: Column 'Final Views' in {realSheet} for vlogger")
            break
        print("Completed: formatting of vlogger")
        ##Reporter Data Formation Starts
        print("Started: starting formatting of reporters")
        names = otherData['Name']
        names = names.to_list()
        cut = None
        for row, name in enumerate(names):
            if "Reporters / Vloggers" in name:
                cut = row + 1
                break
        if cut is not None:
            reporterData = otherData.iloc[:cut].copy()
            otherData = otherData.iloc[cut:].copy()
        else:
            reporterData = None
            otherData = otherData
            print(f"NOT FOUND: Reporter data as 'Reporters / Vloggers' in the {realSheet} for reporter")
        if (cut is not None) and (reporterData is not None):
            foundReporter, formattedReporterData = dateLogicMaker3000(realSheet, "Reporter", reporterData, formattedReporterData)
            if foundReporter == False:
                print(f"NOT FOUND: Column 'Final Views' in {realSheet} for Reporter")
                break
        print("Completed: formatting of reporter")
        ##Anchors Data Formation Starts
        print("Started: starting formatting of anchors")
        keyAnchors = ["Azaad Defence Anchors ", "Azaad Siasat Anchors ","Anchors", "Anchors/Analysts"]
        for keyAnchor in keyAnchors:
            names = otherData['Name']
            names = names.to_list()
            cut = None
            for row, name in enumerate(names):
                if keyAnchor == name:
                    cut = row + 1
                    break
            if cut is not None:
                print(f"FOUND: Anchor data as {keyAnchor} in the {realSheet}")
                if keyAnchor == "Anchors/Analysts":
                    anchorData = otherData
                else:
                    anchorData = otherData.iloc[:cut].copy()
                    otherData = otherData.iloc[cut:].copy()
            else:
                anchorData = None
                otherData = otherData
                print(f"NOT FOUND: Anchor data in the {realSheet} as {keyAnchor}")
                continue
            foundAnchor, formattedAnchorData = dateLogicMaker3000(realSheet, "Anchor", anchorData, formattedAnchorData)
            if foundAnchor == False:
                print(f"NOT FOUND: Column 'Final Views' in {realSheet} for Anchor")
                break
        if foundAnchor == False:
            break
        print("Completed: formatting of anchor")
        ##Analyst Data Formation Starts
        print("Started: starting formatting of analyst")
        foundAnalyst = None
        keyAnalysts = ["Analysts", "Anchors/Analysts"]
        for keyAnalyst in keyAnalysts:
            names = otherData['Name']
            names = names.to_list()
            cut = None
            for row, name in enumerate(names):
                if keyAnalyst == name:
                    cut = row + 1
                    break
            if cut is not None:
                print(f"FOUND: Analyst data as {keyAnalyst} in the {realSheet}")
                if keyAnchor == "Anchors/Analysts":
                    anchorData = otherData
                else:
                    anchorData = otherData.iloc[:cut].copy()
            else:
                anchorData = None
                otherData = otherData
                print(f"NOT FOUND: Analyst data in the {realSheet} as {keyAnalyst}")
                continue
            foundAnalyst, formattedAnalystData = dateLogicMaker3000(realSheet, "Analyst", anchorData, formattedAnalystData)
            if foundAnalyst == False:
                print(f"NOT FOUND: Column 'Final Views' in {realSheet} for Analyst")
                break
        if foundAnalyst is not None and foundAnalyst is False:
            break
        print("Completed: formatting of analyst")
    else:
        print(f"NOT FOUND: Channel {realSheet}")
        break

formattedJournalistData = pd.concat([formattedReporterData, formattedAnchorData, formattedAnalystData], ignore_index=True)

from google.colab import files
from openpyxl.styles import Alignment, Border, Side
from openpyxl.utils import get_column_letter

with pd.ExcelWriter("FormattedData.xlsx", engine="openpyxl") as writer:
    formattedVloggerData.to_excel(writer, sheet_name="Vlogger", index=False)
    formattedJournalistData.to_excel(writer, sheet_name="Journalist", index=False)

    workbook = writer.book
    thin_border = Border(
        left=Side(style="thin"),
        right=Side(style="thin"),
        top=Side(style="thin"),
        bottom=Side(style="thin")
    )

    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]

        max_row = worksheet.max_row
        max_col = worksheet.max_column

        for row in worksheet.iter_rows(min_row=1, max_row=max_row, min_col=1, max_col=max_col):
            for cell in row:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.border = thin_border

        for col in range(1, max_col + 1):
            col_letter = get_column_letter(col)
            worksheet.column_dimensions[col_letter].width = 25

        for row in range(1, max_row + 1):
            worksheet.row_dimensions[row].height = 25

files.download("FormattedData.xlsx")



Enter the number according to the day, month started:
Monday    : 1
Tuesday   : 2
Wednesday : 3
Thursday  : 4
Friday    : 5
Saturday  : 6
Sunday    : 7
Enter number: 1


LOADED SHEET: Azaad Anchors  as Azaad Digital
Started: starting formatting of vloggers
['Name', 'No of Vlogs', 'YT Views', 'Twitter Views', 'TiktokViews', 'FB Views', 'Final Views', 'Ranking', 'No of Vlogs', 'YT Views', 'Twitter Views', 'TiktokViews', 'FB Views', 'Final Views', 'Cahnge %', 'Ranking', 'Highest Values', 'YT Views', 'Twitter Views', 'TiktokViews', 'FB Views', 'Final Views', 'Cahnge %', 'Ranking', 'Highest Value', 'YT Views', 'Twitter Views', 'TiktokViews', 'FB Views', 'Final Views 48 Hrs(8 & 9)', 'Change %', 'Ranking', 'Highest Value', 'No of Shows', 'YT Views', 'Twitter Views', 'TiktokViews', 'FB Views', 'Final Views 48 Hrs(10 & 11)', 'Change %', 'Ranking']

FOUNDED: Column 'Final Views' as Final Views in Azaad Digital starting from 1/Sep/2025
NOT FOUND / MISMATCHED: Key platform column in Azaad Digital 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>